# Tutorial 1: Creating and Solving Factor Graphs

## API Version: NavAbilitySDK.jl

In this tutorial you will learn how to create, solve, and explore factor graphs, as well as what variables, factors, and priors are.

## What is a Factor Graph

Wikipedia tells us a factor graph is a bipartite graph representing the factorization of a function.
 
If you haven't taken some advanced statistics classes you are probably wondering: Bipartite graph? Factorization? I just want to navigate my robot.
For robotics, factor graphs are the common language used to describe your robot's estimation problem (the function) in a way both humans and computers can understand. 
The estimation problem can be anything from robot localization, structure from motion, calibration to full SLAM with parametric or non-parametric measurements and beliefs.
A factor graph is a graphical model with two types of nodes, variables and factors, connected by edges between the variables and factors (bipartite graph). 
Variables represent the unknown random variables in the estimation problem, such as vehicle or landmark positions, sensor calibration parameters, and more.
Factors represent the algebraic interaction between particular variables using stochastic models, for example wheel odometry between poses, which is depicted with edges. 
A graph of variables and factors is the factorization of the function describing your system, and effectively represents a breakdown of the complex problem describing your robot navigation; 
for example, the graph models the position and orientation (pose) of your robot at any given time relative to landmarks. 
This factorization allows us to solve the optimization (a.k.a. inference) problem for all variables given every measurement described by the factors.
See Caesar.jl docs [Graph Concepts](https://juliarobotics.org/Caesar.jl/latest/concepts/concepts/#Graph-Concepts) for more detail.

### Let's build a factor graph

First, we will need some packages: 
- `NavAbilitySDK` - Access NavAbility Cloud factor graph features from Julia.
- `LinearAlgebra` and `UUIDs` - Standard Julia libraries that will be useful.

In [ ]:
# install packages in case they are not installed
import Pkg; Pkg.add("NavAbilitySDK");

In [ ]:
using NavAbilitySDK

The first thing to do is setup a client-context to talk with the NavAbility Platform:

In [ ]:
# you need a unique userId:robotId, and can keep using that across all tutorials
userId = "Guest"
robotId = "SDKjl_"*(string(uuid4())[1:4])

# also create a client connection
client = NavAbilityHttpsClient()

In [ ]:
# You'll need a unique session number each time you run a new graph
sessionId = "Tutorial1_"*(string(uuid4())[1:4])
# context is the object to use below
context = Client(userId,robotId,sessionId)

## Variables and Factors 

Variables, denoted as the larger nodes in the figure below, represent state variables of interest such as vehicle or landmark positions, sensor calibration parameters, and more. Variables are likely hidden values that are not directly observed, but we want to estimate them from observed data and at least some minimal algebra structure from probabilistic measurement models.

Factors, the smaller nodes in the figure below, represent the algebraic interaction between particular variables, which is captured through edges. Factors must adhere to the limits of probabilistic models – for example conditional likelihoods capture the likelihood correlations between variables; while priors (unary to one variable) represent absolute information to be introduced. 

`NavAbilitySDK.jl` provides variables and factors useful to robotics.
We start with a `Pose2` variable, i.e. position and orientation in two dimensions.
Call `addVariable` with a label `x0` and type `Pose2` to add variables to the factor graph

In [ ]:
# lets collect all the async responses and wait at the end
resultIds = Task[]
# addVariable and keep the transaction ID
push!(resultIds, 
  addVariable(client, context, Variable("x0", :Pose2))
);

We now have a factor graph with one variable, but to solve it we need some additional information. 
In this example, we need the estimated starting point of our robot.
We use unary factors called priors to represent absolute information to be introduced. 
In this case we use `PriorPose2`, as our variable type is also `Pose2`.
Since factors represent a probabilistic interaction between variables, we need to specify the distribution our factor will represent. Here we use `FullNormal` which is a [multivariate normal distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution). 

Let's create a `PriorPose2` unary factor with zero mean and a covariance matrix of (`diagm([0.05,0.05,0.01].^2)`):

$\Sigma = \begin{bmatrix} 0.0025 & 0.0 &  0.0 \\ 0.0 & 0.0025 & 0.0 \\ 0.0 &  0.0 &  0.0001 \end{bmatrix}$

In [ ]:
f = Factor("x0f1", "PriorPose2", ["x0"], 
PriorPose2Data(
  Z=FullNormal(
    [0.0, 0.0, 0.0], 
    diagm([0.05, 0.05, 0.01].^2))))

push!(resultIds, addFactor(client, context, f));

In [ ]:
# lets wait to make sure all nodes were added
waitForCompletion(client, resultIds; expectedStatuses=["Complete"])

We can look at the factor graph we have so far using the generated link to the NavAbility WebApp

In [ ]:
println("https://app.navability.io/cloud/graph/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

The prior is now connected to the variable, `x0`, but it is not initialized yet. Automatic initialization of variables depends on how the factor graph model is constructed. So far, `x0` has not been initialized.

**Graph-based Initialization**

Why is `x0` not initialized? Since no other variable nodes have been 'connected to' (or depend) on `x0` and future intentions of the user are unknown, the initialization of `x0` is deferred until the latest possible moment.  The NavAbility Platform assumes that the new variables and factors can be initialized when they are solved for the first time.

By delaying initialization of a new variable (say `x0`) until a second newer uninitialized variable (say `x1`) depends on `x0`, the `Caesar.jl` algorithms wait to initialize `x0` with the more information from surrounding variables and factors.  Also, note that graph-based initialization of variables is a local operation based only on the neighboring nodes – global inference over the entire graph is shown later in this tutorial.

**Robot Odometry - Relative Factor**

Next, we want to add an odometry factor that connects our two robot poses `x0` and `x1` together to form a chain.
Here we use a relative factor of type `Pose2Pose2` with a measurement from pose `x0` to `x1` of (x=1.0,y=0.0,θ=pi/2); the robot drove 1 unit forward (in the x direction).
Similarly to the prior we added above, we use an `FullNormal` distribution to represent the odometry with mean and covariance:

$\mu =(x=1, y=0, \theta=\frac{\pi}{2})$

$\Sigma = \begin{bmatrix} 0.01 & 0.0 &  0.0 \\ 0.0 & 0.01 & 0.0 \\ 0.0 &  0.0 &  0.0001 \end{bmatrix}$


In [ ]:
# add x1
push!(resultIds, 
  addVariable(client, context, Variable("x1", :Pose2))
)

# add odometry measurement between x0 and x1
f = Factor("x0x1f1", "Pose2Pose2", ["x0","x1"], 
  Pose2Pose2Data(
    Z=FullNormal(
      [1.0, 0.0, pi/2], 
      diagm([0.1, 0.1, 0.01].^2))
  )
)
push!(resultIds, addFactor(client, context, f));

Lets look at the factor graph again

In [ ]:
println("https://app.navability.io/cloud/graph/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

The structure of the graph has now been updated to two variable nodes and two factors.

### Solving

We now have a graph we can solve using the Multi-Modal iSAM (MM-iSAM) algorithm. 
The default solver will perform non-parametric inference/state-estimation over our newly created graph.

In [ ]:
push!(resultIds,
  solveSession(client, context)
);

# Give it a few seconds JIT compiling during first run.

### Results

The NavAbility WebApp allows visualization of the belief state over any of the variables.  Also try with Belief and Distribution buttons to see more of the underlying posterior marginal belief estimates.  The results will show as the solve completes.

In [ ]:
println("https://app.navability.io/cloud/map/?userStartsWith=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

**What is happening**

The figure shows the position and orientation (red forward) for poses `x0` and `x1`. As well as the covariance ellipse. 
Since the solver used was non-parametric, the covariance ellipse is based on a best Gaussian distribution fit of the full belief.
A few other functions are also handy for interacting with the factor graph, for instance `getVariable` returns the full variable.


Or if you are interested in the suggested Parametric Point Estimate (PPE).

In [ ]:
vars = getVariables(client, context; detail=SUMMARY) |> fetch
lbls = vars .|> x->x["label"]
ppes = vars .|> x->float.(x["ppes"][1]["suggested"])

println.(lbls .=> ppes);

**Parametric point estimates and beliefs**

A PPE can be the maximum or mean of the belief. 
If the belief is a normal distribution, both correspond with its mean. 
However, care should be taken with using PPEs when beliefs might be non-parametric, for example, in a multimodal belief with two peaks, max corresponds with the maximum of the two peaks while the mean will fall somewhere in between them. 
In non-parametric cases, it is better to work with the full belief obtained by the Kernel Density Estimate (KDE).  
Kernel Density Estimation is a non-parametric way to estimate the probability density function of a random variable.
With the default solver, a full probability density function is always available and can be visualized as shown by the distribution plot feature in the NavAbility WebApp.
Non-parametric solutions will be discussed in more detail in tutorial 2.

In [ ]:
println("https://app.navability.io/cloud/map/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

More plotting options exist, depending on how you are accessing the data.  See the [section on Plotting in the Caesar docs](https://juliarobotics.org/Caesar.jl/latest/concepts/2d_plotting/) for additional detail.

### Adding more poses and a point landmark

So far we worked with the `Pose2` factor type. 
Among others, `NavAbilitySDK` also provides the `Point2` variable and `Pose2Point2BearingRange` factor types, which we will use to represent a landmark sighting in our factor graph.
We will add a landmark `l1` with bearing range measuremet of bearing=$(\mu=0,\sigma=0.03)$ range=$(\mu=0.5,\sigma=0.1)$ and continue our robot trajectory by driving around in a square.

In [ ]:
# add one landmark
push!(resultIds,
  addVariable(client, context, Variable("l1", :Point2)))

# add three more poses
for x in ["x2"; "x3"; "x4"]
  push!(resultIds,
    addVariable(client, context, Variable(x, :Pose2)))
end

## add Factors
F = [
  # add landmark observation measurement and
  Factor("x0l1f1", "Pose2Point2BearingRange", ["x0","l1"], 
  Pose2Point2BearingRangeData(
    bearing = Normal(0, 0.03), 
    range = Normal(0.5, 0.1))),
    
  # odometry measurements between poses
  Factor("x1x2f1", "Pose2Pose2", ["x1","x2"], 
  Pose2Pose2Data(
    Z=FullNormal(
      [1.0, 0.0, pi/2], 
      diagm([0.1, 0.1, 0.01].^2))
  )),
  Factor("x2x3f1", "Pose2Pose2", ["x2","x3"], 
  Pose2Pose2Data(
    Z=FullNormal(
      [1.0, 0.0, pi/2], 
      diagm([0.1, 0.1, 0.01].^2))
  )),
  Factor("x3x4f1", "Pose2Pose2", ["x3","x4"], 
  Pose2Pose2Data(
    Z=FullNormal(
      [1.0, 0.0, pi/2], 
      diagm([0.1, 0.1, 0.01].^2))
  ))
]

[push!(resultIds, addFactor(client, context, f)) for f in F];

Lets go look at the factor graph now, again using the WebApp

In [ ]:
println("https://app.navability.io/cloud/graph/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

We now have a longer odometry chain with one landmark sighting, let's solve the factor graph again so we can have a look at the results.

In [ ]:
# lets wait to make sure all the new additions are ready
waitForCompletion(client, resultIds; expectedStatuses=["Complete"], maxSeconds=180)
# then start a single solve
println("Start a new solve...")
push!(resultIds,
  solveSession(client, context)
);

The solve will take a bit of time.  Just keep watching the geometric visualization, which will automatically update as more of the solution is published

In [ ]:
println("https://app.navability.io/cloud/map/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

### Adding a Loop Closure

As expected, the robot continued its square trajectory to end off where it started. 
To illustrate a loop closure, we add another bearing range sighting to from pose `x4` to landmark `l1`, solve the graph and plot the new results: 

In [ ]:
# add a loop closure landmark observation
f = Factor("x4l1f1", "Pose2Point2BearingRange", ["x4","l1"], 
  Pose2Point2BearingRangeData(
    bearing = Normal(0, 0.03), 
    range = Normal(0.5, 0.1))
)
push!(resultIds, addFactor(client, context, f));
#

# lets wait to make sure all the new additions are ready
waitForCompletion(client, resultIds; expectedStatuses=["Complete"])

# then start a single solve
println("Start new solve...")
push!(resultIds,
  solveSession(client, context)
);

Lets go look at the final results again, which now includes the loop closure.  Use the 'Show Belief', 'Show Distribution' buttons along the bottom for more visual information.  You can also trigger new solves (as long as the Global Filter fields are properly set in the far right filter menu).

Use the WebApp hamburger menu on the left to navigate between the graph and geometric map visualization.  You can also use the Global Filter menu on the right to set which information to visualize.

In [ ]:
println("https://app.navability.io/cloud/graph/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

In [ ]:
println("https://app.navability.io/cloud/map/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

## Next Steps

Tutorial 2 will give an introduction to non-parametric solutions.

## Additional Resources

### Variable representation and factor algebraic operations

The variables used in this tutiorail, `Point2` and `Pose2`, are represented as points on manifolds and all algebraic operations, inference/optimization are also performed on manifold.
For more information on how manifolds are used in Ceaser.jl, refer to the [manifold section in the documentation](https://juliarobotics.org/Caesar.jl/latest/concepts/using_manifolds/).

### Custom Variables and Factors

In most scenarios, the existing variables and factors should be sufficient for most robotics applications. 
Caesar however, is extensible and allows you to easily incorporate your own variable and factor types for specialized applications.

Have a look at the Caesar documentaton if you are interested in creating custom vairables, factors (or priors)

- [Custom variables](https://juliarobotics.org/Caesar.jl/latest/examples/custom_variables/)
- [Custom prior factors](https://juliarobotics.org/Caesar.jl/latest/examples/basic_definingfactors/)
- [Custom relative factors](https://juliarobotics.org/Caesar.jl/latest/examples/custom_relative_factors/)


### Bayes (Junction) Tree

Inference is performed on the bayes tree see: [Bayes tree principles](https://juliarobotics.org/Caesar.jl/latest/principles/bayestreePrinciples/)